In [1]:
!pip install autogluon.tabular[all]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.8/234.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 3

# Imports #

In [2]:
import numpy as np 
import pandas as pd 

from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularDataset, TabularPredictor

In [3]:
df_train = pd.read_csv("/kaggle/input/playground-series-s4e9/train.csv", index_col="id")
df_test = pd.read_csv("/kaggle/input/playground-series-s4e9/test.csv", index_col="id")

In [4]:
df_train

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
id,,,,,,,,,,,,
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500
...,...,...,...,...,...,...,...,...,...,...,...,...
188528,Cadillac,Escalade ESV Platinum,2017,49000,Gasoline,420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Beige,None reported,Yes,27500
188529,Mercedes-Benz,AMG C 43 AMG C 43 4MATIC,2018,28600,Gasoline,385.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Black,At least 1 accident or damage reported,Yes,30000
188530,Mercedes-Benz,AMG GLC 63 Base 4MATIC,2021,13650,Gasoline,469.0HP 4.0L 8 Cylinder Engine Gasoline Fuel,7-Speed A/T,White,Black,None reported,Yes,86900


# Preprocessing #

In [5]:
def clean(df):
    df["fuel_type"] = df["fuel_type"].replace("Plug-In Hybrid", "Hybrid")
    df["clean_title"] = df["clean_title"].fillna("No") #To treat feature as binary

Extract horsepower, liters, and cylinders from `engine` feature.

Extract automatic, manual, or dct (dual clutch transmission) from `transmission` feature. 

In [6]:
def extract_features(df):
    #Engine features
    df["engine_horsepower"] = df["engine"].str.extract(r'(\d+\.\d+|\d+)\s*HP').astype(float)
    df["engine_liters"] = df["engine"].str.extract(r'(\d+\.\d+|\d+)\s*L').astype(float)
    df_cylinders = df['engine'].str.extract(r'(\d+)\s*Cylinder|V(\d+)', expand=False)
    df['engine_cylinders'] = df_cylinders[0].fillna(df_cylinders[1]) #Combine both regex searches
    
    #Transmission features
    df["transmission_automatic"] = df["transmission"].str.contains(r'Automatic|A/T|AT', case=False)
    df["transmission_manual"] = df["transmission"].str.contains(r'Manual|M/T|MT', case=False)
    df["transimssion_dct"] = df["transmission"].str.contains('Dual', case=False)

Bundle feature extraction and cleaning into one pipeline

In [7]:
def preprocess(df_train, df_test):
    df = pd.concat([df_train, df_test], axis=0)
    clean(df)
    extract_features(df)
    df_train = df.loc[df_train.index]
    df_test = df.loc[df_test.index]
    return df_train, df_test

In [8]:
df_train, df_test = preprocess(df_train, df_test)

In [9]:
df_train.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,engine_horsepower,engine_liters,engine_cylinders,transmission_automatic,transmission_manual,transimssion_dct
id,,,,,,,,,,,,,,,,,,
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200.0,172.0,1.6,4,True,False,False
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999.0,252.0,3.9,8,True,False,False
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900.0,320.0,5.3,8,True,False,False
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000.0,420.0,5.0,8,False,False,True
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500.0,208.0,2.0,4,True,False,False


# Modeling #

First, create holdout set for validation. Random seed is to keep consistent OOF predictions throughout different notebooks.

In [10]:
RANDOM_SEED = 31415

df_train, df_val = train_test_split(df_train, test_size=0.1, random_state=RANDOM_SEED)

In [11]:
#Convert for autogluon
ds_train = TabularDataset(df_train)
ds_val = TabularDataset(df_val)
ds_test = TabularDataset(df_test)


In [12]:
#10 hour time limit
model = TabularPredictor(label="price", eval_metric="rmse").fit(ds_train, presets="best_quality", time_limit=3600*10)

No path specified. Models will be saved in: "AutogluonModels/ag-20240917_025818"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Jun 27 20:43:36 UTC 2024
CPU Count:          4
Memory Avail:       30.12 GB / 31.36 GB (96.1%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets o

In [13]:
model.leaderboard(ds_val)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost_BAG_L2,-71187.786024,-72607.071460,root_mean_squared_error,115.427907,337.287519,18078.977614,0.391277,0.852518,501.476453,2,True,38
1,CatBoost_r9_BAG_L2,-71195.319422,-72637.081649,root_mean_squared_error,115.659450,337.809852,17821.129326,0.622820,1.374850,243.628165,2,True,48
2,CatBoost_r177_BAG_L2,-71204.649777,-72578.937052,root_mean_squared_error,115.282389,337.138226,17826.747016,0.245760,0.703224,249.245855,2,True,44
3,WeightedEnsemble_L3,-71208.285268,-72537.765834,root_mean_squared_error,127.748861,376.737563,21077.069272,0.007586,0.003599,0.655330,3,True,50
4,WeightedEnsemble_L2,-71295.964523,-72653.571441,root_mean_squared_error,44.360888,80.967544,8688.927425,0.010180,0.003531,0.677216,2,True,34
5,CatBoost_r13_BAG_L1,-71328.946000,-72814.222501,root_mean_squared_error,3.295590,3.505539,2018.902127,3.295590,3.505539,2018.902127,1,True,23
6,LightGBMXT_BAG_L2,-71337.678837,-72776.239351,root_mean_squared_error,115.544931,338.971747,17671.301968,0.508302,2.536745,93.800807,2,True,35
7,XGBoost_BAG_L2,-71359.051349,-72957.842301,root_mean_squared_error,116.780269,339.913059,17903.955821,1.743640,3.478057,326.454660,2,True,41
8,CatBoost_r137_BAG_L1,-71376.831993,-72876.335861,root_mean_squared_error,0.884054,1.396226,1117.377760,0.884054,1.396226,1117.377760,1,True,21
9,CatBoost_r177_BAG_L1,-71379.442039,-72869.663287,root_mean_squared_error,1.214457,1.229832,448.395301,1.214457,1.229832,448.395301,1,True,12


In [14]:
predictions = model.predict(ds_test)

In [15]:
predictions

id
188533    17707.371094
188534    81218.015625
188535    57209.531250
188536    31460.142578
188537    29345.843750
              ...     
314218    26967.785156
314219    50342.238281
314220    21060.966797
314221    15592.314453
314222    35853.500000
Name: price, Length: 125690, dtype: float32

In [16]:
predictions.to_csv("submission.csv")